In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import holidays
from datetime import date,datetime

In [2]:
PATH_TO_DATA = 'data/predict_future_sales'
items = pd.read_csv(os.path.join(PATH_TO_DATA, 'items.csv'))
item_categories = pd.read_csv(os.path.join(PATH_TO_DATA, 'item_categories.csv'))
shops = pd.read_csv(os.path.join(PATH_TO_DATA, 'shops.csv'))
test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(PATH_TO_DATA, 'sample_submission.csv'))
sales_train = pd.read_pickle('data/sales_train_dwncast.pkl')

In [3]:
%%time
#sales_train['date'] = pd.to_datetime(sales_train['date'], format="%d.%m.%Y")
sales_train['date'] = sales_train['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
sales_train['year'] = sales_train['date'].apply(lambda x: x.year).astype(np.int16)
sales_train['month'] = sales_train['date'].apply(lambda x: x.month).astype(np.int16)
sales_train['day'] = sales_train['date'].apply(lambda x: x.day).astype(np.int16)

CPU times: user 1min 34s, sys: 772 ms, total: 1min 35s
Wall time: 1min 33s


In [4]:
#Lets get the Russian holidays
ru_holiday =  holidays.RU()

#defining a function to get whether the given day was an holiday or not
def is_holiday(x):
    if x in ru_holiday:
      return 1 
    return 0
#for e.g.
#df['date'].apply(is_holiday)

In [5]:
sales_train['is_holiday'] = sales_train['date'].apply(is_holiday).astype(np.int16)

In [6]:
#chekcing the new columns
sales_train = sales_train.sort_values(by='date', ascending=True)
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day,is_holiday
49800,2013-01-01,0,18,5823,2500.0,1.0,2013,1,1,1
29784,2013-01-01,0,27,5573,849.0,1.0,2013,1,1,1
35476,2013-01-01,0,7,1006,399.0,1.0,2013,1,1,1
8330,2013-01-01,0,19,17707,899.0,1.0,2013,1,1,1
57384,2013-01-01,0,14,19548,149.0,1.0,2013,1,1,1


In [7]:
# Print all the holidays in UnitedKingdom in year 2018 
for ptr in holidays.Russia(years = 2013).items(): 
    print(ptr) 

(datetime.date(2013, 1, 1), 'Новый год')
(datetime.date(2013, 1, 2), 'Новый год')
(datetime.date(2013, 1, 3), 'Новый год')
(datetime.date(2013, 1, 4), 'Новый год')
(datetime.date(2013, 1, 5), 'Новый год')
(datetime.date(2013, 1, 6), 'Новый год')
(datetime.date(2013, 1, 7), 'Православное Рождество')
(datetime.date(2013, 1, 8), 'Новый год')
(datetime.date(2013, 2, 23), 'День защитника отечества')
(datetime.date(2013, 3, 8), 'День женщин')
(datetime.date(2013, 5, 1), 'Праздник Весны и Труда')
(datetime.date(2013, 5, 9), 'День Победы')
(datetime.date(2013, 6, 12), 'День России')
(datetime.date(2013, 11, 4), 'День народного единства')


We see different dates for holidays in the holiday calender in the internet. 
Lets assume the library to be correct for now

sales_train.info()

In [10]:
#sales_train.to_pickle('data/sales_train_fe.pkl')

### The below datasets
- #### item_categories contains item_category_name & item_category_id
- #### As shops contains shop_name & shop_id
#### We will merge items with sales_train as it contains item_category_id , another feature


In [11]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [8]:
#Lets drop item_name as its only the description of the item
items.drop('item_name', axis=1, inplace=True)
items.head()

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


In [13]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 2 columns):
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2)
memory usage: 346.5 KB


In [9]:
def downcast_datatypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df
#converting float64 to float32 and int64,int32 to int16

In [10]:
items = downcast_datatypes(items)

In [11]:
len(sales_train['item_id'].unique())

21807

In [83]:
#sales_train = pd.read_pickle('data/sales_train_fe.pkl')

In [12]:
#merging the items to sales_train
#The items dataset has 22170 unique item ids, whereas the sales training dataset has only 21806 unique item ids
#sales_train = sales_train.merge(items, on='item_id', how='right')
sales_train_merge = sales_train.join(items.set_index('item_id'), on='item_id')

In [13]:
sales_train_merge.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day,is_holiday,item_category_id
49800,2013-01-01,0,18,5823,2500.0,1.0,2013,1,1,1,35
29784,2013-01-01,0,27,5573,849.0,1.0,2013,1,1,1,2
35476,2013-01-01,0,7,1006,399.0,1.0,2013,1,1,1,67
8330,2013-01-01,0,19,17707,899.0,1.0,2013,1,1,1,19
57384,2013-01-01,0,14,19548,149.0,1.0,2013,1,1,1,40


In [51]:
#sales_train['item_category_id'] = sales_train['item_category_id'].astype(np.int16)

In [14]:
sales_train_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 49800 to 2928850
Data columns (total 11 columns):
date                datetime64[ns]
date_block_num      int16
shop_id             int16
item_id             int16
item_price          float32
item_cnt_day        float32
year                int16
month               int16
day                 int16
is_holiday          int16
item_category_id    int16
dtypes: datetime64[ns](1), float32(2), int16(8)
memory usage: 112.0 MB


In [15]:
sales_train_merge['item_category_id'].isnull().values.any()

False

In [24]:
#As expected no null values are present in the item_category_id

In [16]:
sales_train_merge.to_pickle('data/sales_train_fe.pkl')

In [ ]:
#Lets update the test data

In [17]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [29]:
test_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 6 columns):
ID                  214200 non-null int16
shop_id             214200 non-null int16
item_id             214200 non-null int16
item_category_id    214200 non-null int16
year                214200 non-null int16
month               214200 non-null int16
dtypes: int16(6)
memory usage: 2.5 MB


In [21]:
test_merge = test.join(items.set_index('item_id'), on='item_id')

In [30]:
test_merge.head()

,ID,shop_id,item_id,item_category_id,year,month
0,0,5,5037,19,2015,11
1,1,5,5320,55,2015,11
2,2,5,5233,19,2015,11
3,3,5,5232,23,2015,11
4,4,5,5268,20,2015,11


In [23]:
test_merge['item_category_id'].isnull().values.any()

False

In [24]:
test_merge['year'] = 2015#.astype(np.int16)
test_merge['month'] = 11#.astype(np.int16)

In [28]:
test_merge = downcast_datatypes(test_merge)

In [31]:
test_merge.to_pickle('data/sales_test_fe.pkl')